<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TO-DO-to-complete-NLP-prototype" data-toc-modified-id="TO-DO-to-complete-NLP-prototype-1">TO DO to complete NLP prototype</a></span></li><li><span><a href="#Explore-NLP-methods/vizs-with-champaign-restaurant-review-data" data-toc-modified-id="Explore-NLP-methods/vizs-with-champaign-restaurant-review-data-2">Explore NLP methods/vizs with champaign restaurant review data</a></span><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-2.1">Import packages</a></span></li><li><span><a href="#Get-data" data-toc-modified-id="Get-data-2.2">Get data</a></span></li><li><span><a href="#Data-preview" data-toc-modified-id="Data-preview-2.3">Data preview</a></span></li><li><span><a href="#Tokenize" data-toc-modified-id="Tokenize-2.4">Tokenize</a></span></li><li><span><a href="#Make-dictionary-&amp;-corpus" data-toc-modified-id="Make-dictionary-&amp;-corpus-2.5">Make dictionary &amp; corpus</a></span></li><li><span><a href="#Get-Similarities" data-toc-modified-id="Get-Similarities-2.6">Get Similarities</a></span></li><li><span><a href="#Visualizations" data-toc-modified-id="Visualizations-2.7">Visualizations</a></span></li></ul></li></ul></div>

# TO DO to complete NLP prototype

- query similarities using combinations
- put edge data into networkx
- make plotly network viz
- add a few plotly node features
- clean up/refactor code

# Explore NLP methods/vizs with champaign restaurant review data

## Import packages

In [54]:
#sqlalchemy and pandas for data 
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pandas as pd
#spacy for tokenization
from spacy.lang.en import English # Create the nlp object
import spacy
#gensim for similarity
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.similarities.docsim import MatrixSimilarity,Similarity
#itertools for getting similarity edges
#networkx for organizing similarities
#plotly for visualization

## Get data

In [55]:
postgres_db_params = {'drivername': 'postgres',
                'database':'restaurants',
               'username': 'michaelkranz',
               'password': 'helloworld',
               'host': 'localhost',
               'port': 5432}

postgres_db_url = URL(**postgres_db_params)
engine = create_engine(postgres_db_url)

In [56]:
review_df = pd.read_sql(
    con=engine.connect(),
    sql='''
    SELECT *
    FROM champaign_restaurant_reviews
    ''')

## Data preview

In [57]:
review_df.head(2)

,business_id,name,review_id,user_id,text,stars,date
0,9A1C1f0m4nQltQrOOTl-Kw,Orange & Brew,m4AXzV9l14iFBd9DRdM82w,6X0i-oGUbh5DZdTHzFuKfg,The building is lovely. The remodel after But...,1.0,2013-12-07 02:26:13
1,VHsNB3pdGVcRgs6C3jt6Zg,Dublin O'Neil's,A-yKlSLEQQcoHR5q2lCyHg,Yximlvn0cfb3yVDaLuXDxw,LOVE LOVE LOVE this place! I'm a bit of a suck...,5.0,2013-08-03 19:59:56


In [58]:
review_df.head(2).text.values[0]

'The building is lovely.  The remodel after Buttita\'s is as good as one can do when turning a lovely restaurant into a sports bar-style restaurant (though why one would want to do that is beyond me).  And that\'s where the good stuff stops.\nIn short, the service and all things related to it were glacially slow; the food, when it arrived, was mediocre at best -- and missing key elements (one whole order, and other parts).  This restaurant cannot possibly last if they don\'t figure out how to greet and serve customers, or get the food that was ordered actually to the tables.\nThe full story.\nUpon arrival, we found a vacant hostess stand, then a long walk down the hall to the dining room along which we saw no one who works there.  Luckily, we ran into friends who assured us that we could seat ourselves.\nAfter waiting at our table for some 15 minutes, we used a bit of self-help, and accosted a waitress to ask for some menus.  After another 5-7 minutes another waitress finally showed up

## Tokenize

In [59]:
def tokenize_text(text_str,nlp_obj):
    '''
    use spacy to separate text into words
    (ie tokenization)
    and return the lemmatization 
    (ie feet for footing and foot)
    for only nouns and adjectives
    
    TODO: refine methodology
    '''
    spacy_doc = nlp_obj(text_str)
    
    tokenized_doc = [
        token.lemma_
        for token in spacy_doc
        if token.pos_ in ("NOUN","ADJ")
        ]
    
    return tokenized_doc
    #return spacy_doc
        

## Make dictionary & corpus

In [60]:
nlp = spacy.load('en_core_web_sm')

In [61]:
#TODO: combine reviews in SQL to scale
#TODO: sciktilearn?

In [62]:
reviews_tokenized = (
    review_df
    #.head(2)
    .groupby('business_id')
    .text
    .apply(lambda x: ' '.join(x))
    .apply(tokenize_text,nlp_obj=nlp)
)

In [63]:
reviews_dictionary = Dictionary(reviews_tokenized)

In [64]:
reviews_dictionary.num_docs

701

In [65]:
review_df.business_id.unique().shape

(701,)

In [66]:
#corpus
reviews_corpus = [reviews_dictionary.doc2bow(doc) for doc in reviews_tokenized]

In [67]:
#tfidf with document being each restaurant and corpus being all restaurants
reviews_tfidf_model = TfidfModel(reviews_corpus)

In [68]:
reviews_tfidf_docs = [reviews_tfidf_model[review] for review in reviews_corpus]

## Get Similarities

In [69]:
# Function which returns subset or r length from n 
# https://docs.python.org/2/library/itertools.html
from itertools import combinations,groupby

In [72]:
#similarity indices for each doc
similarity_indices = MatrixSimilarity(reviews_tfidf_docs)

In [88]:
test = ['dff','fdf']

In [89]:
import inspect

In [92]:
#def make_combinations(self,num_):
    #    combinations(self)

In [159]:
make_combinations(test)

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [161]:
{'similarity':y for x,y in make_combinations(test)}

{'similarity': 4}

In [181]:
def make_edges_df(node_list,
                  node_names=['x','y']):
    edges = combinations(node_list,2)
    edges_df = pd.DataFrame(edges,columns=node_names)
    return edges_df

def _filter_similarities(doc_indices,similarities_to_corpus):
    #get similarity for each pair
    similarities_to_group = {
        (doc1_index,doc2_index):similarities_to_corpus[doc2_index]
        for doc1_index,doc2_index in doc_indices.to_records(index=False)
    }
    return pd.Series(similarities_to_group)

def get_tfidf_similarities(doc_index_series):
    tfidfs_from = reviews_tfidf_docs[doc_index_series.name]
    similarities_to = similarity_indices[tfidfs_from]
    return _filter_similarities(doc_index_series,similarities_to)

In [182]:
node_list = range(len(reviews_tfidf_docs))

In [ ]:
tfidf_similarities = (
    make_edges_df(node_list)
    .groupby('x')
    .apply(get_tfidf_similarities)
    .reset_index('x',drop=True)
)

In [174]:
tfidf_similarities.head(3)

1  2    0.071847
   3    0.046881
   4    0.094364
2  3    0.033973
   4    0.101671
dtype: float64

In [91]:
#gets all combos of documents and sorts by first document index
doc_pairs = sorted(
    combinations(test,r=2),
    key=itemgetter(0)
)

similarity_combo_vals = [
    (idoc_x,idoc_y,doc_xy_sim)
    for idoc_x,group in groupby(doc_pairs)
    for _,idoc_y,doc_xy_sim in filter_similarities()

SyntaxError: unexpected EOF while parsing (<ipython-input-91-d7e9d4cd0975>, line 16)

In [53]:
similarity_indices[]

NameError: name 'similarity_indices' is not defined

In [80]:
test = [[1,[2,2]],[2,[3,3]]]

In [83]:
[(key,values) for key,values in test]

[(1, [2, 2]), (2, [3, 3])]

In [84]:
def filter_similarities(similarities_to_corpus,doc_index_pairs):
    #get similarity for each pair
    similarities_to_group = [
        (doc1_index,doc2_index,similarities_to_corpus[doc2_index])
        for doc1_index,doc2_index in doc_index_pairs
    ]
    return similarities_to_group

In [86]:
filter_similarities(similarity_indices[reviews_tfidf_docs[0]],[(0,1),(0,2)])

[(0, 1, 0.056968044), (0, 2, 0.19020963)]

In [ ]:
#get all combos
#sort combos by first element
#for each first element: 
    #get similarities for all second elements
    #add to tuple
    #(docx,docy,sim)

#

In [ ]:
#similarity to each restaurant (ie doc)
fiesta_similarity = (
    pd.DataFrameame(similarity_indices[fiesta_tfidf])
    .assign(index=doc_id)
)

In [ ]:
#get index:name mappings
doc_mapping = (
    review_df
    .groupby('business_id')
    ['name']
    .apply(lambda x: x.unique()[0])
)

#ie switch index to key and token str to object
token_mapping = {
    i:token 
    for token,i in reviews_dictionary.token2id.items()
} 

In [ ]:
#for orange and brew and bevande coffee (doc indices 0 and 172):
#doc_id = 0
all_bag_of_words_list = []
for doc_id in range(len(reviews_corpus)):
    bag_of_words = (
        pd.DataFrame(
            {"frequency":dict(reviews_corpus[doc_id]),
             "tf_idf":dict(reviews_tfidf_docs[doc_id]),
             "business":doc_mapping[doc_id],
             "word":token_mapping
            }
        )
        .set_index(['business','word'])
    )
    all_bag_of_words_list.append(bag_of_words)                                 

In [ ]:
all_bag_of_words_df = pd.concat(all_bag_of_words_list)

In [2]:
test = [1,2,3,4]

In [ ]:
all_similarities_list = []
for doc_id in range(len(reviews_corpus)):
    #doc example
    doc = reviews_corpus[doc_id]
    doc_tfidf = reviews_tfidf_model[doc]
    similarities = similarity_indices[doc_tfidf]
    
    all_similarities_list.append(similarities)    

## Visualizations

In [ ]:
import plotly.express as px

In [ ]:
restaurants = ['Orange & Brew','Bevande Coffee']

In [ ]:
plot_df = (
    all_bag_of_words_df
    .loc[restaurants]
    .unstack('business')
    .reset_index()
)

plot_df.columns = [
    '_'.join([x for x in column if len(x)>0]) 
    for column in plot_df.columns
]

In [ ]:
plot_df.head(2)

In [ ]:
hover_data_list = [
    'word',
    'frequency_Orange & Brew',
    'frequency_Bevande Coffee'
]

#x_str = "frequency_Orange & Brew"
#y_str = "frequency_Bevande Coffee"

x_str = "tf_idf_Orange & Brew"
y_str = "tf_idf_Bevande Coffee"

In [ ]:
plot_df.columns

In [ ]:
plot_df.word.unique().shape

In [ ]:
fig = px.scatter(plot_df,
                 x=x_str, 
                 y=y_str, 
                 #color="species",
                 #size='petal_length', 
                 hover_data=hover_data_list)
fig.show()

In [ ]:
#ratings

#location